## Export ArcGIS Online or Portal for ArcGIS Users to CSV
This script allows a user to login to a portal and export a CSV list of all users and their corresponding items within it

#### First, let's import the ArcGIS Python API & Other Required Python Modules

In [ ]:
from arcgis.gis import GIS
from arcgis.gis import RoleManager
import getpass
import csv
import os

#### Now, let's define some input parameters
A box will appear allowing you to enter your password. When running interactively, you will be prompted to enter a password. Hit the Enter key to continue.

In [ ]:
# Enter the URL to your ArcGIS Online Organization or Portal for ArcGIS
portalURL = 'https://org.maps.arcgis.com'
# Enter the username & password for an Administrator in your Organization
username = 'jeremy'
password = getpass.getpass()
outputFile = r'C:\temp\output.csv'

#### Next, let's find out how many users are in our Organization
*Note*: the default user search is for 100 users, we are overriding that to 1000 users below

In [ ]:
gis = GIS(portalURL, username, password)
allUsers = gis.users.search(max_users=1000)
print('Total Portal Users: ' + str(len(allUsers)))

#### Time to export the CSV File

In [ ]:
with open(outputFile, 'w') as output:
    dataWriter = csv.writer(output, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL, lineterminator='\n')
    # Write header row.
    dataWriter.writerow(['Full Name',
                         'Email',
                         'Username',
                         'Date Created',
                         'Role',
                         'Item ID'])
    
    for user in allUsers:
        # Get User's Role
        if user.role == 'org_admin':
            userRole = 'Administrator'
        elif user.role == 'org_publisher':
            userRole = 'Publisher'
        elif user.role == 'org_user':
            userRole = 'User'
        else:
            userRole = RoleManager(gis).get_role(user.role).name
        
        # Get User's Items
        items_by_id = []
        print("Collecting item ids for {}...".format(user.username))
        user_content = user.items()
        # Copy item ids from root folder first
        for item in user_content:
            items_by_id.append(item.itemid)
        # Copy item ids from folders next
        folders = user.folders
        for folder in folders:
            folder_items = user.items(folder=folder['title'])
            for item in folder_items:
                items_by_id.append(item.itemid)
        
        if len(items_by_id) > 0:
            for i, item_by_id in enumerate(items_by_id): 
                dataWriter.writerow([user['fullName'],
                                     user['email'],
                                     user['username'],
                                     time.strftime("%Y-%m-%d",time.gmtime(user['created']/1000)),
                                     userRole,
                                     items_by_id[i]])